# Storage

In [1]:
import numpy as np
import pandas as pd
np.random.seed(1234)
pd.options.display.max_rows=10
pd.__version__

'0.18.0'

In [2]:
df = pd.DataFrame({'A' : range(4), 
                   'B' : 1.0, 
                   'C' : 'foo', 
                   'D' : pd.Timestamp('20130101'), 
                   'E' : 2.0})

In [3]:
df

,A,B,C,D,E
0,0,1.0,foo,2013-01-01,2.0
1,1,1.0,foo,2013-01-01,2.0
2,2,1.0,foo,2013-01-01,2.0
3,3,1.0,foo,2013-01-01,2.0


In [4]:
df.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
E           float64
dtype: object

In [5]:
df._data

BlockManager
Items: Index(['A', 'B', 'C', 'D', 'E'], dtype='object')
Axis 1: RangeIndex(start=0, stop=4, step=1)
FloatBlock: slice(1, 7, 3), 2 x 4, dtype: float64
IntBlock: slice(0, 1, 1), 1 x 4, dtype: int64
DatetimeBlock: slice(3, 4, 1), 1 x 4, dtype: datetime64[ns]
ObjectBlock: slice(2, 3, 1), 1 x 4, dtype: object

## Why do we have this arrangement?

In [34]:
arr = np.random.randn(1000000, 10)

In [35]:
df = DataFrame(arr)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
0    1000000 non-null float64
1    1000000 non-null float64
2    1000000 non-null float64
3    1000000 non-null float64
4    1000000 non-null float64
5    1000000 non-null float64
6    1000000 non-null float64
7    1000000 non-null float64
8    1000000 non-null float64
9    1000000 non-null float64
dtypes: float64(10)
memory usage: 76.3 MB


Constructing direct from a np.array

In [36]:
%timeit DataFrame(arr)

10000 loops, best of 3: 86.1 µs per loop


Ufunc operation

In [37]:
result_blocked = df.sum()
result_blocked

0     -17.104726
1    -834.119332
2    2210.913286
3     364.359127
4    1171.210326
5     380.694123
6     668.276510
7    -409.029518
8    -690.061819
9   -1069.483088
dtype: float64

In [38]:
%timeit df.sum()

10 loops, best of 3: 84.3 ms per loop


Construct a columnar layout

In [39]:
d = [ Series(v.copy()) for c, v in df.iteritems() ]

In [40]:
def f(d):
    return pd.Series([ e.sum() for e in d])
result_columnar = f(d)
result_columnar

0     -17.104726
1    -834.119332
2    2210.913286
3     364.359127
4    1171.210326
5     380.694123
6     668.276510
7    -409.029518
8    -690.061819
9   -1069.483088
dtype: float64

In [41]:
np.allclose(result_columnar, result_blocked)

True

## We are *cache* friendly!

In [42]:
%timeit f(d)

10 loops, best of 3: 37.9 ms per loop


Can't we just use *views*?

In [43]:
d = [ pd.Series(v) for c, v in df.iteritems() ]

But this is NOT friendly

In [44]:
%timeit f(d)

10 loops, best of 3: 127 ms per loop


Construct a column like DataFrame

In [45]:
%timeit [ pd.Series(v.copy()) for c, v in df.iteritems() ]

10 loops, best of 3: 64.8 ms per loop


Construct a frame from columnar, with blocking

In [46]:
%timeit DataFrame({c:pd.Series(v) for c, v in df.iteritems()})

10 loops, best of 3: 64 ms per loop


## Further Reading

http://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays/